In [1]:
import os
os.environ["JAVA_HOME"] = r"C:\Program Files\JAVA\jdk-22"

In [ ]:
import torch

import mowl
mowl.init_jvm('10g')
from mowl.datasets import Dataset
from org.semanticweb.owlapi.apibinding import OWLManager
import java.io

from src.utils import *
from src.elmodule import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

In [4]:
dataset_name = 'OWL2DL-1'
# dataset_name = 'family'

In [5]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [ ]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']         

    subsumption_results = []
    membership_results = []
    link_prediction_results = []
    
    for _ in range(5):                                                                                                     

        train_manager = OWLManager.createOWLOntologyManager()
        test_manager = OWLManager.createOWLOntologyManager()
        val_manager = OWLManager.createOWLOntologyManager()

        train_ontology = train_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{file_name}_train.owl')) # we add noise to train
        test_ontology = test_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{dataset_name}_test.owl'))
        val_ontology = val_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{dataset_name}_val.owl'))

        train_ont = preprocess_ontology_el(train_ontology)
        test_ont = preprocess_ontology_el(test_ontology)
        valid_ont = preprocess_ontology_el(val_ontology)

        dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
        
        model = ElModel(dataset, 
                        module_name='box2el', 
                        dim=200, 
                        margin=0.1, 
                        batch_size=4096*8, 
                        test_batch_size=32, 
                        epochs=500, 
                        learning_rate=0.001,
                        device=device)
        
        model._train()
        
        print(f'{file_name}:')
        print('Membership:')
        metrics_membership = model._eval('membership')
        print('Subsumption:')
        metrics_subsumption = model._eval('subsumption')
        print('Link Prediction:')
        metrics_link_prediction = model._eval('link_prediction')
        print()

        subsumption_results.append(metrics_subsumption)
        membership_results.append(metrics_membership)
        link_prediction_results.append(metrics_link_prediction)

    avg_metrics_subsumption = [np.mean(values) for values in zip(*subsumption_results)]
    avg_metrics_membership = [np.mean(values) for values in zip(*membership_results)]
    avg_metrics_link_prediction = [np.mean(values) for values in zip(*link_prediction_results)]

    save_results(avg_metrics_subsumption, avg_metrics_membership, avg_metrics_link_prediction, f'models/results/box2el/{file_name}.txt')

In [1]:
from src.utils import get_experiments
import rdflib

In [2]:
experiments = get_experiments('family')

In [4]:
for experiment in experiments[1:]: 
    
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']

    g_train = rdflib.Graph()
    g_train.parse(f'datasets/{dataset_name}_train.owl', format='xml')

    g_noise = rdflib.Graph()
    g_noise.parse(f'datasets/noise/{file_name}.owl', format='xml')

    g_train += g_noise
    g_train.serialize(destination=f'datasets/{file_name}_train.owl', format='xml')